### The Backstory and the Setup

Your friend Kelly is coming to London on vacation and you want to help her plan out her journey- which spots to see and what routes to take to do it in the most efficient way.

The data set that we are going to use is the [London tube/overgound locations](https://tfl.gov.uk/info-for/open-data-users/). To save time, we have already pre-downloaded and cleaned the data set for you. If you are interested in doing this yourself, you can register at the tfl website, get an api key, download the file, and clean it by removing all `\t`, `\n` and converting dos line endings to unix.

In [ ]:
import pandas as pd
stations = pd.read_hdf('./datasets/london_stations.h5')

# quick sanity check
stations.head()

You can do some pretty cool stuff with a pandas dataframe, including filtering rows using conditionals like so:

In [ ]:
# only look at the rows of the stations dataframe where the station_name column is 'Aldgate Station'
stations[stations.station_name == 'Aldgate Station']

In [ ]:
# only look at the rows of the stations dataframe where the coordinate-x column is greater than 0
stations[stations['coordinate-x'] > 0].head()

### Hold on...

Notice how we use `stations.station_name` in the first filter but `stations['coordinate-x'] > 0` in the second filter?

Why doesn't this work?

    stations[stations.coordinate-x > 0]

In [ ]:
# let's rename the column names so that there is no `-` in them
stations = stations.rename(columns={'coordinate-x': 'x', 'coordinate-y': 'y', 'station_name': 'name'})

# sanity check. also, a new function called tail(). Guess what it does?
stations[stations.y > 51.5].tail()

### Fun Distractions

Let's take a quick break from Pandas, and talk about where you are going to tell your friend to go.

We could just find the longitude/latitude coordinates for popular landmarks in London, but what's the fun in that? [Geo-caching](https://youtu.be/1YTqitVK-Ts) is fun, and you want to introduce your friend to it. We are going to generate a list of coordinates for your friend to visit.

In [ ]:
from collections import namedtuple
from random import uniform

# here is a cool python feature. We could have also used pygeoif.geometry.Point, or just had an un-named tuple/list
Coordinates = namedtuple('Coord', ['x', 'y'])

# Our London playground is going to be between -1° to 1° longitude and 52° to 51° latitude
targets_list = [
    Coordinates(x=uniform(-1, 1), y=uniform(51,52)) 
    for ii in range(200)
]
print(targets_list[:5])

In [ ]:
# what's a pandas Series?
targets = pd.Series(targets_list)
print(targets.head())

In [ ]:
# you can also access Series and Dataframes using their index
targets.iloc[0]

In [ ]:
# let's separate out x and y into two different columns
# the apply function maps a function over a pandas Series or Dataframe
targets = pd.DataFrame({
    'x': targets.apply(lambda p: p.x),
    'y': targets.apply(lambda p: p.y)
})
targets.head()

### Data cleaning and more sanity checks

In [ ]:
# let's make sure that our data is squeeky clean
stations.isnull().any()

In [ ]:
# another way to do this is to see if any columns have missing rows
stations.count()

### Let the computations begin!

In [ ]:
# let's assume the world is flat for the moment...
from scipy.spatial import KDTree
# http://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html
stationsTree = KDTree(stations[['x', 'y']])
targetsTree = KDTree(targets[['x', 'y']])

In [ ]:
# what's a good way to check how this works?
# well if you have a small enough neighborhood radius, no stations should be neighbors with anybody but themselves
print(stationsTree.count_neighbors(stationsTree, r=0.001))
print(len(stations))

In [ ]:
# huh?
print(stationsTree.count_neighbors(stationsTree, r=0.00001))

In [ ]:
stationsTree.query_pairs(r=0.001)

In [ ]:
# oh...
stations.iloc[[11, 132, 288, 289]]

### GeoCaching targets that are close to subway stations 

In [ ]:
# what target points are within 1000m of a subway station?

# let's assume that every 0.0001° is 7m
# https://en.wikipedia.org/wiki/Geographic_coordinate_system#Expressing_latitude_and_longitude_as_linear_units
radius = 1000.0 / 7 * 0.0001

for idx, target in targets.iterrows():
    close_by_stations_idx = stationsTree.query_ball_point([target.x, target.y], r=radius)
    if close_by_stations_idx:
        stations_name = stations.iloc[close_by_stations_idx]['name'].tolist()
        # string formatting!
        print('Target {} ({:.2f}, {:.2f}) is close to stations {}'.format(idx, target.x, target.y, stations_name) )

What else can you try to do? Here is a couple suggestions:

Access the google maps api to get transportation times between different subway stations (or also add in say 10 geocaching targets). Then try to figure out what is the quickest way to say visit all 10 targets!

There are a couple ways to do this, and it also depends on your requirements (eg: do you want to finish where you started). Take a look at the different algorithms for the [shortest path problem](https://en.wikipedia.org/wiki/Shortest_path_problem), and the [travelling salesman problem](https://simple.wikipedia.org/wiki/Travelling_salesman_problem).

Try to implement a particular algorithm yourself! (maybe learn to set up the algorithm in a separate file, and import it here)

We found the shortest distance assuming that the world was flat- which is actually a pretty accurate approximation within London. Look into calculating the distance based on a globe-shaped geometry. It could be pretty easy to use a different distance function, but then you wouldn't be able to use the KDTrees. If you wanted to keep using that, you want to turn all coordinates to a vector to represent where they are located in space. Then the standard cartesian distance function would still work and so you can keep using KDTrees. One library for this is [proj4](https://github.com/jswhit/pyproj). Or for this usecase, you could just write a quick function to set that up yourself as well.

Once you start getting into graph theory problems like TSP, maybe it would be cool to try setting up a Neo4j database and visualizing the data that way! (definitely out of the scope of this tutorial though)

Lastly, [here](http://nbviewer.jupyter.org/github/mqlaql/geospatial-data/blob/master/Geospatial-Data-with-Python.ipynb) are some really cool map visualizations.